In [1]:
!pip install lightly==1.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.6/647.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.0 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=03e0751c4c97798b22886ee7f3e658107d34dac5bbfe6cbbfa41ad0c66fac1cc
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [2]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
import torchvision
import torch.nn as nn
from lightly.models.modules.heads import SimCLRProjectionHead

class SimCLRModel(nn.Module):
    def __init__(self):    
        super(SimCLRModel, self).__init__()
        # create a ResNet backbone and remove the classification head
        # The dilation parameters are given, as backbone will be reused with deeplabv3+ model
        backbone = torchvision.models.resnet50(pretrained=True, replace_stride_with_dilation =[False, True, True])
        self.initial = nn.Sequential(*list(backbone.children())[:4])
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
        self.avgpool = backbone.avgpool

        hidden_dim = backbone.fc.in_features
        self.projection_head = SimCLRProjectionHead(hidden_dim, hidden_dim, 128)
    
    def backbone_forward(self, x):
        # Pass input through Backbone layers
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x
        
    def forward(self, x):
        x = self.backbone_forward(x)
        x = self.avgpool(x).flatten(start_dim=1)
        return self.projection_head(x)         

In [4]:
# dino pretrained model wieghts
dino_resnet50 = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50')

# custom model
model = SimCLRModel()

# number of backbone layers  = 318
from collections import OrderedDict
test_soln = OrderedDict(zip(list(model.state_dict().keys())[:318], list(dino_resnet50.state_dict().values())[:318]))
model.load_state_dict(test_soln, strict=False)

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_resnet50_pretrain/dino_resnet50_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_resnet50_pretrain.pth
100%|██████████| 90.0M/90.0M [00:00<00:00, 164MB/s] 
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0

_IncompatibleKeys(missing_keys=['projection_head.layers.0.weight', 'projection_head.layers.1.weight', 'projection_head.layers.1.bias', 'projection_head.layers.1.running_mean', 'projection_head.layers.1.running_var', 'projection_head.layers.3.weight', 'projection_head.layers.4.weight', 'projection_head.layers.4.bias', 'projection_head.layers.4.running_mean', 'projection_head.layers.4.running_var'], unexpected_keys=[])

In [5]:
torch.save(model.state_dict(), "dino_resnet50.pt")